In [4]:
import tensorflow as tf
import time
from datetime import timedelta
import math
import numpy as np
import os
import glob
import sys
import scipy.misc
from scipy.ndimage import imread
import timeit

In [5]:
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    def getid(s):
        return int(''.join(c for c in s[3:] if c.isdigit()))
        
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort(key=getid)
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)
        

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    One-hot encoding converts categorical labels to binary values
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    print(os.listdir())
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1 
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)
    
def get_batch(X, y, batch_size):
    """
    Return minibatch of samples and labels
  
    :param X, y: samples and corresponding labels
    :parma batch_size: minibatch size
    :returns: (tuple) X_batch, y_batch
    """
    # Random indices for the samples

    indices = np.random.randint(y.shape[0]-1, size= batch_size)

    X_batch = X[indices, :]
    y_batch = y[indices, :]
  
    return X_batch, y_batch

# Load the data

data_root_path = 'HW2_data/'

X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
y_train = one_hot(y_train).T
X_train = X_train.T
X_train = (X_train - np.mean(X_train,axis=0)) / np.std(X_train,axis=0) # Normalize data 


# Loading test data 
X_test = get_images(data_root_path + 'test').T

print("Data loading done.")

['.DS_Store', '.ipynb_checkpoints', 'CNN.ipynb', 'HW2_data', 'HW2_data.zip', 'HW2_data_t', 'img_aug1.ipynb', 'img_aug2.ipynb', 'img_aug3.ipynb', 'Notes.txt', 'test_submit.csv', 'train.py', 'train_hw2.py']
{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """


Loaded 10000/45000
Loaded 20000/45000
Loaded 30000/45000
Loaded 40000/45000
Data loading done.


In [6]:
val_size = len(X_train)-3000

In [7]:
X_train1= X_train[0:val_size,:]
y_train1= y_train[0:val_size]
X_val = X_train[val_size:,:] 
y_val = y_train[val_size:]

In [8]:
X_train1 = X_train1.reshape(len(X_train1),32,32,3)

In [9]:
batch_size = 64
num_classes = 10
img_size = 32
num_channels = 3

In [10]:
tf.reset_default_graph()
session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x')
## labels
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)
is_training = tf.placeholder(tf.bool)

In [11]:
# Convolutional layers 
with tf.name_scope("conv"):
    Wconv1 = tf.get_variable("Wconv1", shape=[5, 5, 3, 32])
    bconv1 = tf.get_variable("bconv1", shape=[32])
    Wconv2 = tf.get_variable("Wconv2", shape=[5, 5, 32, 64])
    bconv2 = tf.get_variable("bconv2", shape=[64])
    
    # Summarizing convolution layers
    tf.summary.histogram("Wconv1", Wconv1)
    tf.summary.histogram("bconv1", Wconv1)
    tf.summary.histogram("Wconv2", Wconv2)
    tf.summary.histogram("bconv2", bconv2)

# Fully connected layers 
with tf.name_scope("fc"):
    W1 = tf.get_variable("W1", shape=[3136, 1024])
    b1 = tf.get_variable("b1", shape=[1024])
    W2 = tf.get_variable("W2", shape=[1024, 10])
    b2 = tf.get_variable("b2", shape=[10]) 
    
    # Summarizing last fully connected layer
    tf.summary.histogram("W2", Wconv2)
    tf.summary.histogram("b2", bconv2)


# Forward propagation
conv1 = tf.nn.conv2d(x, Wconv1, strides=[1, 1, 1, 1], padding='SAME') + bconv1
relu1 = tf.nn.relu(conv1)
conv2 = tf.nn.conv2d(relu1, Wconv2, strides=[1, 2, 2, 1], padding='VALID') + bconv2
relu2 = tf.nn.relu(conv2)
maxpool = tf.layers.max_pooling2d(relu2, pool_size=(2,2), strides=2)
maxpool_flat = tf.reshape(maxpool,[-1,3136])
# Spatial Batch Normalization Layer (trainable parameters, with scale and centering)
bn1 = tf.layers.batch_normalization(inputs=maxpool_flat, center=True, scale=True, training=is_training)
affine1 = tf.matmul(bn1, W1) + b1
affine1_flat = tf.reshape(affine1,[-1,1024])
bn2 = tf.layers.batch_normalization(inputs=affine1, center=True, scale=True, training=is_training)
relu2 = tf.nn.relu(bn2)
drop1 = tf.layers.dropout(inputs=relu2, training=is_training)
affine2 = tf.matmul(drop1, W2) + b2
affine2_flat = tf.reshape(affine2,[-1,3136])

predict = tf.layers.batch_normalization(inputs=affine2, center=True, scale=True, training=is_training)
y_pred = tf.nn.softmax(predict)

In [12]:
y_pred_cls = tf.argmax(y_pred, axis=1)

with tf.name_scope("cost"): # cross-entropy scope
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=predict,labels=y_true)
    cost = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("cost", cost)
    
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

summ = tf.summary.merge_all() # Merging the summary 

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 1e-3
end_learning_rate = 5e-3
decay_steps = 10000

learning_rate = tf.train.polynomial_decay(starter_learning_rate, global_step,
                                          decay_steps, end_learning_rate,
                                          power=0.5)

exp_learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               100000, 0.96, staircase=True)                 

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost, global_step=global_step)
                      

In [13]:
x_valid_batch = X_val 
y_valid_batch = y_val #= get_batch(X_train, y_train, batch_size)
x_valid_batch = x_valid_batch.reshape((3000, 32, 32, 3))

In [14]:
def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

In [15]:
def train(num_epochs = 10):
    step = 0
    for epoch in range(num_epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        for i in range(int(math.ceil(X_train.shape[0]/batch_size))):
            
            x_batch, y_true_batch = get_batch(X_train1, y_train1, batch_size)
            # Training batches 
            feed_dict_tr = {x: x_batch,
                           y_true: y_true_batch, is_training: True}
            
            # Validation batches 
            feed_dict_val = {x: x_valid_batch,
                              y_true: y_valid_batch, is_training: False}
            
            # Optimizing 
            session.run(optimizer, feed_dict=feed_dict_tr)
            
            # Adding the summary 
            if (i % 5) == 0:
                s = session.run(summ, feed_dict = feed_dict_tr)
                writer.add_summary(s, step)
            
            if (i % 200) == 0:
                # Showing progress every 200 iters
                val_loss = session.run(cost, feed_dict=feed_dict_val)
                show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)     
            
            step += 1

In [16]:
session = tf.Session()

In [17]:
session.run(tf.global_variables_initializer())

path = "/tmp/cifarcnn/3" # Change if want to keep the old board 
writer = tf.summary.FileWriter(path)
writer.add_graph(session.graph)

In [15]:
try:
    with tf.device("/cpu:0") as dev:
        train(num_epochs=10)
except tf.errors.InvalidArgumentError:
    print("no gpu found, please use Google Cloud if you want GPU acceleration") 

Training Epoch 1 --- Training Accuracy:  90.6%, Validation Accuracy:  19.3%,  Validation Loss: 2.276
Training Epoch 1 --- Training Accuracy:  54.7%, Validation Accuracy:  46.2%,  Validation Loss: 1.631
Training Epoch 1 --- Training Accuracy:  57.8%, Validation Accuracy:  59.5%,  Validation Loss: 1.240
Training Epoch 1 --- Training Accuracy:  60.9%, Validation Accuracy:  63.1%,  Validation Loss: 1.105
Training Epoch 2 --- Training Accuracy:  67.2%, Validation Accuracy:  58.7%,  Validation Loss: 1.184
Training Epoch 2 --- Training Accuracy:  76.6%, Validation Accuracy:  60.8%,  Validation Loss: 1.132
Training Epoch 2 --- Training Accuracy:  79.7%, Validation Accuracy:  65.4%,  Validation Loss: 1.035
Training Epoch 2 --- Training Accuracy:  71.9%, Validation Accuracy:  66.7%,  Validation Loss: 0.992
Training Epoch 3 --- Training Accuracy:  85.9%, Validation Accuracy:  66.9%,  Validation Loss: 0.996
Training Epoch 3 --- Training Accuracy:  81.2%, Validation Accuracy:  67.6%,  Validation Lo

In [55]:
X_test = (X_test - np.mean(X_test,axis=0)) / np.std(X_test,axis=0)

In [56]:
X_test.shape

(5000, 3072)

In [57]:
pred = session.run(y_pred_cls, feed_dict = {x: X_test.reshape((5000,32,32,3)), is_training: True})

In [58]:
pred

array([7, 1, 4, ..., 9, 1, 1])

In [59]:
classes = ('airplane','automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [62]:
file= open("test_submit.csv", "w")
file.write("id,label\n")


for i in range(len(pred)):
    row = str(i+1) + ","+ classes[pred[i]] + '\n'
    file.write(row)
file.close()